# Optiver Quiz - Solution

This quiz can be found on the [Optiver's webpage](https://www.optiver.com/eu/en/job-opportunities/eu-517702).

This is the original text of the quiz:

**Can you solve this puzzle?**

Three players A, B, C play the following game. First, A picks a real number between 0 and 1 (both inclusive), then B picks a number in the same range (different from A’s choice) and finally C picks a number, also in the same range, (different from the two chosen numbers). We then pick a number in the range uniformly randomly. Whoever’s number is closest to this random number wins the game. Assume that A, B and C all play optimally and their sole goal is to maximise their chances of winning. Also assume that if one of them has several optimal choices, then that player will randomly pick one of the optimal choices.

- a) If A chooses 0, then what is the best choice for B?
- b) What is the best choice for A?
- c) Can you write a program to figure out the best choice for the first player when the game is played among four players?

### Strategy of the implementation

- I apply a *brute force* approach where I evaluate all possible combinations of choices for all players. 
- I assume the game is played between $0$ and $100$ (as these value can be interpreted as percentages) and I assume that $1\%$ is the minimum distance two players' choices can differ by.
- I represent the game decision tree as nested Python dictionaries, where each key of the dictionary corresponds to a decision of position for a given player.
- Loosely speaking, the algorithm of the *optiver_tree()* function (which is the main function to solve the quiz) basically performs a dynamic optimization: for every choice of the players from the first to the last, it checks the optimal solution of the last player who has to choose and then propagates this optimal choice back to the preceeding player, and so on to the first player. The resulting decision tree represents the optimal choices for all players.
- The optimal choice for any given player is obviously the choice that maximizes her probabilities of winning.
- Several enhancements are possible, like taking into account the symmetry of the [0,1] interval, or avoiding the *brute-force* approach, since with only 5 players would take too long to evaluate (with 4 players the code below evaluates $94\,109\,400$ possible choices, while with 5 players it has to evaluate a whopping $9\,034\,502\,400$ of possibile combinations!).

### Function to evaluate a single combination of positions

In [1]:
import pprint
game = [i for i in range(101)]

In [2]:
def evaluate_position(tocheck, game_board=game):
    ''' 
    It takes in an array with players' positions and returns the probabilities 
    of winning the game for each player
    '''
    results = []
    orders = []
    final_results = []
    tocheck_sorted = sorted(tocheck)
    assert len(list(set(tocheck)))==len(tocheck_sorted)
    for idx, x in enumerate(tocheck_sorted):
        if x == min(tocheck_sorted): 
            results.append(x + (tocheck_sorted[idx+1]- x)/2)
            orders.append(x)
        elif x == max(tocheck_sorted): 
            results.append(len(game_board)-1-x + abs(tocheck_sorted[idx-1]- x)/2)
            orders.append(x)
        else:               
            results.append(abs(tocheck_sorted[idx-1]- x)/2 + abs(tocheck_sorted[idx+1]- x)/2)
            orders.append(x)
    for x in tocheck:
        for idx, y in enumerate(orders):
            if x == y: final_results.append(results[idx])
    return final_results

evaluate_position((25,50,75), game)

[37.5, 25.0, 37.5]

### Optiver tree recursive function

In [3]:
def solve_optiver_quiz(num_players, fix_player=0, fix_player_value=0):
    return optiver_tree(num_players, 1, 0, {}, [], [], fix_player, fix_player_value)[0] 

def optiver_tree(num_players, game_step=1, player_num=0, tree={}, game_keys=[], max_val=[], 
                 fix_player=0, fix_player_value=0):
    assert num_players > 1, 'Number of Player has to be greater than one.'
    assert num_players <= 4, 'It takes too long to evaluate with more than 4 players!'
    if not max_val: max_val=[-float('inf')]*num_players
    game_val = []
    opt_val = []
    count = 1
    run_next = True
    for i in range(0, 101, game_step):
        if fix_player == (player_num+1) and i != fix_player_value: run_next = False
        if not i in game_keys and run_next:
            game_keys.append(i)
            if player_num == num_players-1:
                game_val = evaluate_position(game_keys)
            else:
                tree_result, game_val = optiver_tree(num_players, game_step, player_num+1, {}, game_keys
                                                     , [], fix_player, fix_player_value)
            if game_val:
                if game_val[player_num] > max_val[player_num]:
                    tree = {}
                    if player_num != num_players-1:
                        tree[i] = tree_result
                    else: tree[i] = game_val
                    max_val[player_num] = game_val[player_num]
                    opt_val = game_val
                    count = 1
                elif game_val[player_num] == max_val[player_num]:
                    if player_num != num_players-1:
                        tree[i] = tree_result
                    else: tree[i] = game_val
                    opt_val = [a+b for a,b in zip(game_val, opt_val)]
                    count +=1
            del game_keys[player_num]
        run_next = True
    opt_val = [a/count for a in opt_val]
    return tree, opt_val

### Answers to questions a), b) and c)

- a) If A chooses 0, then what is the best choice for B?
- b) What is the best choice for A?
- c) Can you write a program to figure out the best choice for the first player when the game is played among four players?

#### Question A

In [4]:
question_a = solve_optiver_quiz(3, 1, 0)

In [5]:
print('If A chooses 0, then the best choice for B is: {}'.format(list(question_a[0].keys())))

If A chooses 0, then the best choice for B is: [67]


- a) We see that if A chooses 0, then the best choice for B is $\frac{2}{3}$ (or approx. $67\%$).

#### Question B

In [6]:
question_b = solve_optiver_quiz(3)

In [7]:
print('The best choice for A is: {}'.format(list(question_b.keys())))

The best choice for A is: [25, 75]


- b) The best choices for A (with three players in total) are $\frac{1}{4}$ and $\frac{3}{4}$.

We can see below the total tree of optimal moves for all three players (every key of the dictionary represents a move for one of the players, the list contained at the end of each entry of the nested dictiorary represents the probabilities of winning for each player, represented in percentage terms). So, for example:

{25: {75: {26: [25.5, 49.5, 25.0]

means that player A chooses $25\%$, player B $75\%$, and player C $26\%$, and the probabilities of winning are $25.5\%$ for player A, $49.5\%$ for player B, and $25\%$ for player C.

In [8]:
pprint.pprint(question_b)

{25: {75: {26: [25.5, 49.5, 25.0],
           27: [26.0, 49.0, 25.0],
           28: [26.5, 48.5, 25.0],
           29: [27.0, 48.0, 25.0],
           30: [27.5, 47.5, 25.0],
           31: [28.0, 47.0, 25.0],
           32: [28.5, 46.5, 25.0],
           33: [29.0, 46.0, 25.0],
           34: [29.5, 45.5, 25.0],
           35: [30.0, 45.0, 25.0],
           36: [30.5, 44.5, 25.0],
           37: [31.0, 44.0, 25.0],
           38: [31.5, 43.5, 25.0],
           39: [32.0, 43.0, 25.0],
           40: [32.5, 42.5, 25.0],
           41: [33.0, 42.0, 25.0],
           42: [33.5, 41.5, 25.0],
           43: [34.0, 41.0, 25.0],
           44: [34.5, 40.5, 25.0],
           45: [35.0, 40.0, 25.0],
           46: [35.5, 39.5, 25.0],
           47: [36.0, 39.0, 25.0],
           48: [36.5, 38.5, 25.0],
           49: [37.0, 38.0, 25.0],
           50: [37.5, 37.5, 25.0],
           51: [38.0, 37.0, 25.0],
           52: [38.5, 36.5, 25.0],
           53: [39.0, 36.0, 25.0],
           54: [39.5

#### Question C

In [9]:
question_c = solve_optiver_quiz(4)

In [10]:
print('The best choice for A is: {}'.format(list(question_c.keys())))

The best choice for A is: [17, 83]


- c) The best choices for A (when the game is played among four players) are $17\%$ and $83\%$. 

In the cell below you can see the full optimal tree for all four players. These are the optmial moves for all players A, B, C, and D:

- player A: the optimal moves are $17\%$ and $83\%$;
- player B is basically symmetric to player A: player B chooses $83\%$ if player A chooses $17\%$, and player B chooses $17\%$ if player A chooses $83\%$  
- player C: the optimal choice is in the middle of the interval, i.e. $50\%$;
- player D: the optimal choices are: "just a bit to the left of the player who chooses $17\%$ " (in my implementation this means $16\%$); "just a bit to the right of the player who chooses $83\%$" (i.e. $84\%$); and everywhere in the middle between player A and player B (i.e. between $17\%$ and $83\%$, obviously excluding $17\%$, $83\%$ and $50\%$ beacuse are chosen by the previous players)

#### Optimal tree with 4 players

In [11]:
pprint.pprint(question_c)

{17: {83: {50: {16: [17.0, 33.5, 33.0, 16.5],
                18: [17.5, 33.5, 32.5, 16.5],
                19: [18.0, 33.5, 32.0, 16.5],
                20: [18.5, 33.5, 31.5, 16.5],
                21: [19.0, 33.5, 31.0, 16.5],
                22: [19.5, 33.5, 30.5, 16.5],
                23: [20.0, 33.5, 30.0, 16.5],
                24: [20.5, 33.5, 29.5, 16.5],
                25: [21.0, 33.5, 29.0, 16.5],
                26: [21.5, 33.5, 28.5, 16.5],
                27: [22.0, 33.5, 28.0, 16.5],
                28: [22.5, 33.5, 27.5, 16.5],
                29: [23.0, 33.5, 27.0, 16.5],
                30: [23.5, 33.5, 26.5, 16.5],
                31: [24.0, 33.5, 26.0, 16.5],
                32: [24.5, 33.5, 25.5, 16.5],
                33: [25.0, 33.5, 25.0, 16.5],
                34: [25.5, 33.5, 24.5, 16.5],
                35: [26.0, 33.5, 24.0, 16.5],
                36: [26.5, 33.5, 23.5, 16.5],
                37: [27.0, 33.5, 23.0, 16.5],
                38: [27.5, 33.5, 2